# Apprentissage supervisé avec Python - Credit Scoring

## II. Apprentissage supervisé : Données hétérogènes

In [1]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [59]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import KFold,cross_val_score,cross_validate
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import time

### Question 1

### Chargement des données et préparation

- Importation du jeu de données

In [6]:
data = pd.read_csv('./credit.data', sep='\t')

- Création de sous-ensemble des variables numériques

Pour cela, nous allons séparer les colonnes numériques des colonnes catégorielles

In [7]:
col_num = [1, 2, 7, 10, 13, 14]  # Colonnes numériques
col_str = [0, 3, 4, 5, 6, 8, 9, 11, 12]  # Colonnes catégoriques

In [10]:
col_str + col_num

[0, 3, 4, 5, 6, 8, 9, 11, 12, 1, 2, 7, 10, 13, 14]

Nous allons maintenant remplacer les "?" par NaN dans toutes les colonnes

In [11]:
for i in col_str + col_num:
    data.iloc[:, i] = data.iloc[:, i].apply(lambda x: np.nan if x == '?' else x)


Ensuite nous allons convertir les colonnes numériques en float, puis convertir le DataFrame en numpy array

In [13]:
data.iloc[:, col_num] = data.iloc[:, col_num].astype(float)
data_np = data.values  

- Tranformation du jeu de données

In [45]:
X=data.iloc[:,col_num]
Y=data.iloc[:,-1]

In [17]:
X_num = data.iloc[:, col_num].astype(float).values

- Suppression des individus contenant des $\textbf{NaN}$

In [46]:
data.dropna(axis=0, inplace=True)

- Analyse des propriétés des données

In [47]:
print(f"Taille de l'échantillon (lignes, colonnes) : {data.shape}")

Taille de l'échantillon (lignes, colonnes) : (652, 16)


In [48]:
pourcentage_negative=100*np.sum(Y=='-')/len(Y)
pourcentage_positive=100*np.sum(Y=='+')/len(Y)
print('Pourcentage positif {0:.2f}%, Pourcentage négatif {1:.2f}%'.format(pourcentage_positive,pourcentage_negative))

Pourcentage positif 45.25%, Pourcentage négatif 54.75%


- Binarisation 

In [49]:
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)  

$\textbf{Comparaison des résultats obtenus à l'aide des différents algorithmes}$

In [50]:
clfs = {
    'NB': GaussianNB(),
    'RF': RandomForestClassifier(n_estimators=200, random_state=1),
    'BAG': BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=1), n_estimators=200, random_state=1),
    'ADA': AdaBoostClassifier(n_estimators=200, random_state=1),
    'ET': ExtraTreesClassifier(n_estimators=200, random_state=1),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'CART': DecisionTreeClassifier(criterion='gini', random_state=1),
    'ID3': DecisionTreeClassifier(criterion='entropy', random_state=1),
    'Stumb': DecisionTreeClassifier(criterion='gini', max_depth=1, random_state=1),
    'MLP': MLPClassifier(hidden_layer_sizes=(20, 10), random_state=1),
    'XGB': XGBClassifier(n_estimators=200, random_state=1)
}


In [51]:
# Fonction d’évaluation des classifieurs
def run_classifieurs(X, Y):
    kf = KFold(n_splits=10, shuffle=True, random_state=1)
    for i in clfs:
        clf = clfs[i]
        start = time.time()
        cv_results = cross_validate(clf, X, Y, cv=kf, scoring=['accuracy', 'precision', 'roc_auc'])
        end = time.time()

        print("Algorithm: {0}".format(i))
        print(" - Accuracy: {0:.3f} +/- {1:.3f}".format(np.mean(cv_results['test_accuracy']), np.std(cv_results['test_accuracy'])))
        print(" - AUC: {0:.3f} +/- {1:.3f}".format(np.mean(cv_results['test_roc_auc']), np.std(cv_results['test_roc_auc'])))
        print(" - Training time: {0:.3f}s\n".format(end - start))


In [52]:
run_classifieurs(X,Y)

Algorithm: NB
 - Accuracy: 0.713 +/- 0.040
 - AUC: 0.801 +/- 0.025
 - Training time: 0.389s

Algorithm: RF
 - Accuracy: 0.784 +/- 0.036
 - AUC: 0.846 +/- 0.043
 - Training time: 10.747s

Algorithm: BAG
 - Accuracy: 0.768 +/- 0.035
 - AUC: 0.836 +/- 0.043
 - Training time: 16.363s

Algorithm: ADA
 - Accuracy: 0.762 +/- 0.030
 - AUC: 0.820 +/- 0.051
 - Training time: 9.608s

Algorithm: ET
 - Accuracy: 0.762 +/- 0.025
 - AUC: 0.839 +/- 0.044
 - Training time: 8.058s

Algorithm: KNN
 - Accuracy: 0.695 +/- 0.033
 - AUC: 0.730 +/- 0.038
 - Training time: 0.412s

Algorithm: CART
 - Accuracy: 0.681 +/- 0.074
 - AUC: 0.682 +/- 0.076
 - Training time: 0.292s

Algorithm: ID3
 - Accuracy: 0.732 +/- 0.066
 - AUC: 0.733 +/- 0.069
 - Training time: 0.196s

Algorithm: Stumb
 - Accuracy: 0.744 +/- 0.047
 - AUC: 0.724 +/- 0.045
 - Training time: 0.171s

Algorithm: MLP
 - Accuracy: 0.639 +/- 0.048
 - AUC: 0.669 +/- 0.036
 - Training time: 1.757s

Algorithm: XGB
 - Accuracy: 0.755 +/- 0.042
 - AUC: 0.835 

Le Random Forest (RF) est le meilleur modèle car il offre un bon équilibre entre précision ($78.4\%$) et AUC ($84.6\%$), malgré un temps d'entraînement plus long.

Les modèles comme CART, Stump et MLP sont moins efficaces, leurs AUC sont trop faibles

Toutefois, Bagging ($83.6\%$), Extra Trees ($83.9\%$) et XGBoost ($83.6\%$) sont de bonnes alternatives

### Normalisation des variables continues

In [54]:
SS=StandardScaler()
SS.fit(X)
Xnorm=SS.transform(X)

In [55]:
run_classifieurs(Xnorm,Y)

Algorithm: NB
 - Accuracy: 0.713 +/- 0.040
 - AUC: 0.801 +/- 0.025
 - Training time: 0.207s

Algorithm: RF
 - Accuracy: 0.784 +/- 0.036
 - AUC: 0.846 +/- 0.043
 - Training time: 11.191s

Algorithm: BAG
 - Accuracy: 0.768 +/- 0.035
 - AUC: 0.837 +/- 0.043
 - Training time: 13.585s

Algorithm: ADA
 - Accuracy: 0.762 +/- 0.030
 - AUC: 0.820 +/- 0.051
 - Training time: 8.829s

Algorithm: ET
 - Accuracy: 0.762 +/- 0.025
 - AUC: 0.839 +/- 0.044
 - Training time: 7.527s

Algorithm: KNN
 - Accuracy: 0.748 +/- 0.056
 - AUC: 0.808 +/- 0.042
 - Training time: 0.162s

Algorithm: CART
 - Accuracy: 0.684 +/- 0.073
 - AUC: 0.685 +/- 0.075
 - Training time: 0.160s

Algorithm: ID3
 - Accuracy: 0.733 +/- 0.065
 - AUC: 0.734 +/- 0.069
 - Training time: 0.231s

Algorithm: Stumb
 - Accuracy: 0.744 +/- 0.047
 - AUC: 0.724 +/- 0.045
 - Training time: 0.112s

Algorithm: MLP
 - Accuracy: 0.785 +/- 0.054
 - AUC: 0.844 +/- 0.056
 - Training time: 6.485s

Algorithm: XGB
 - Accuracy: 0.755 +/- 0.042
 - AUC: 0.835 

Les modèles KNN et les réseaux de neurones MLP ont vu une nette amélioration de leur performance après normalisation, ils sont sensibles à l’échelle des données.

Les modèles basés sur les arbres (Random Forest, XGBoost, Extra Trees, etc.) n'ont pas été impactés par la normalisation.

Le meilleur modèle reste Random Forest (RF) avec une AUC de $0.846$, suivi de MLP ($0.844$) et Extra Trees ($0.839$).

### Question 2

### Traitement des données manquantes

In [57]:
credit = pd.read_csv('credit.data', sep='\t',header=None)

In [58]:
X=credit.iloc[:,:-1].values
Y=credit.iloc[:,-1].values
col_num = [1,2, 7, 10, 13,14]
col_cat = [0,3,4,5,6,8,9,11,12]

Pour les variables catégorielles

In [60]:
X_cat = np.copy(X[:, col_cat])
for col_id in range(len(col_cat)):
    unique_val, val_idx = np.unique(X_cat[:, col_id], return_inverse=True)
    X_cat[:, col_id] = val_idx
    
imp_cat = SimpleImputer(missing_values=0, strategy='most_frequent')
X_cat[:, range(5)] = imp_cat.fit_transform(X_cat[:, range(5)])

Pour les variables numériques

In [61]:
X_num = np.copy(X[:, col_num])
X_num[X_num == '?'] = np.nan
X_num = X_num.astype(float)
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
X_num = imp_num.fit_transform(X_num)

In [68]:
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

### Traitement de varaibles catégorielles

In [63]:
from sklearn.preprocessing import OneHotEncoder
X_cat_bin = OneHotEncoder().fit_transform(X_cat).toarray()

In [64]:
X_cat_bin

array([[0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

### Construction du jeu de données

In [66]:
Xconc = np.concatenate((X_cat_bin,X_num),axis=1)

In [69]:
run_classifieurs(Xconc,Y)

Algorithm: NB
 - Accuracy: 0.840 +/- 0.046
 - AUC: 0.910 +/- 0.035
 - Training time: 0.175s

Algorithm: RF
 - Accuracy: 0.875 +/- 0.040
 - AUC: 0.931 +/- 0.039
 - Training time: 9.626s

Algorithm: BAG
 - Accuracy: 0.871 +/- 0.043
 - AUC: 0.918 +/- 0.048
 - Training time: 14.870s

Algorithm: ADA
 - Accuracy: 0.830 +/- 0.043
 - AUC: 0.885 +/- 0.065
 - Training time: 9.519s

Algorithm: ET
 - Accuracy: 0.865 +/- 0.046
 - AUC: 0.911 +/- 0.040
 - Training time: 7.283s

Algorithm: KNN
 - Accuracy: 0.698 +/- 0.056
 - AUC: 0.730 +/- 0.035
 - Training time: 2.181s

Algorithm: CART
 - Accuracy: 0.805 +/- 0.041
 - AUC: 0.800 +/- 0.047
 - Training time: 0.133s

Algorithm: ID3
 - Accuracy: 0.804 +/- 0.024
 - AUC: 0.798 +/- 0.025
 - Training time: 0.154s

Algorithm: Stumb
 - Accuracy: 0.856 +/- 0.032
 - AUC: 0.861 +/- 0.027
 - Training time: 0.086s

Algorithm: MLP
 - Accuracy: 0.773 +/- 0.065
 - AUC: 0.842 +/- 0.055
 - Training time: 6.634s

Algorithm: XGB
 - Accuracy: 0.871 +/- 0.038
 - AUC: 0.924 +

Les meilleurs modèles sont Random Forest (AUC = 0.931), XGBoost (0.924), Bagging (0.918), Extra Trees (0.911), Naïve Bayes (0.910) avec une excellente capacité de classification.

Les modèles AdaBoost (0.885), MLP (0.842), Stump (0.861) sont aussi performants.

Par contre le modèle KNN (0.730) a une faible distinction des classes et temps d’entraînement élevé.

Le modèle XGBoost est le meilleur choix si l'on recherche un on équilibre entre performance (0.924) et rapidité (2.2s).